In [2]:
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
load_dotenv()
import os 


os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGSMITH_API_KEY")
os.environ['LANGSMITH_PROJECT'] = os.getenv("LANGSMITH_PROJECT")
os.environ['LANGSMITH_TRACING'] = os.getenv("LANGSMITH_TRACING")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

In [3]:
from langchain_groq import ChatGroq 
from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [4]:
project_title = "PayMate: Your Ultimate Payment Companion"
project_description = "PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs."
requirements = ["Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.​", "Enable users to link multiple bank accounts and perform instant fund transfers using UPI.", "Provide users with access to instant micro-loans with minimal documentation.", "Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app."]
requirements_string = "\n".join(requirements)

In [5]:
generate_user_stories_system_prompt = """
**ROLE & OBJECTIVE**

You are an expert Agile Product Owner. Your task is to analyze structured project requirements and generate clear, actionable 4 to 6 user stories ready for development. Return the OUTPUT in the JSON format only. 

---

**TASK BREAKDOWN:**
1. **Analyze** project requirements, objectives, and user needs.
2. **Extract** core features and define user roles.
3. **Decompose** requirements into independent, testable user stories.
4. **Generate** user stories using:
   - **As a [user], I want [goal], so that [value].**
   - Clear, testable **acceptance criteria**.
5. **Prioritize** based on business impact and feasibility.

---

**DESIRED OUTPUT TEMPLATE IN LIST OF JSON**

```
[
    {
        "story_id"="US-001",
        "title"="Manage Shopping Cart",
        "description"="As a shopper, I want to modify my cart before checkout.",
        "acceptance_criteria"=[
            "User can add/remove items.",
            "User can update item quantity.",
            "Cart updates reflect in real-time.",
            "User can see the total price of the cart."
        ]
    }
]
```

---

**GUIDELINES:**
✅ Align with project requirements.
✅ Use concise, clear language.
✅ Ensure user stories are independent and testable.
✅ Prioritize based on business impact.
✅ The **acceptance criteria must be between 2 to 4 points**—no more, no less.  
🚫 Avoid vagueness, missing criteria, or unnecessary technical details.
"""

In [6]:
# revised_user_stories_system_prompt= """
# Role & Objective:
#     As an expert Agile Product Owner, analyze existing user stories and user feedback. Return refined user stories in JSON format, adhering to INVEST principles and Agile best practices.

# Input Requirements:
#     - user_stories (List[Dict]): Existing user stories in the format:
#         [{
#             "story_id": str, 
#             "title": str, 
#             "description": str (template: "As a [user], I want [goal], so that [value]."),
#             "acceptance_criteria": List[str] (2-4 testable items)
#         }]
#     - user_feedback (str): Raw feedback from users to analyze.

# Output Rules:
#     Returns a JSON list with:
#     - "story_id": Original ID for modifications, new ID (e.g., "US-XXX") for additions.
#     - "description": Follows the "As a [user]..." template strictly.
#     - "acceptance_criteria": 2-4 concrete, testable items.
#     - "feedback_applied": Short note justifying changes (required for modifications).

# Guidelines:
#     1. Clarity:
#         - Stories must be atomic (independent) and unambiguous.
#         - Avoid technical jargon (e.g., "use OAuth 2.0" → "log in via Google").
#     2. Feedback Integration:
#         - Explicitly document changes in "feedback_applied".
#         - Preserve original intent unless feedback contradicts it.
#     3. Validation:
#         - Reject redundant or overlapping stories.
#         - Ensure acceptance criteria are measurable (e.g., "loads under 2 seconds").

# Example Execution:
#     >>> Input:
#     {
#         "user_stories": [{
#             "story_id": "US-001",
#             "title": "Checkout",
#             "description": "As a shopper, I want to pay for items, so that I can complete my purchase.",
#             "acceptance_criteria": ["User can enter credit card details."]
#         }],
#         "user_feedback": "Users want PayPal and Apple Pay options."
#     }

#     >>> Output:
#     [{
#         "story_id": "US-001",
#         "title": "Checkout Process",
#         "description": "As a shopper, I want to pay via multiple methods, so that I can complete my purchase flexibly.",
#         "acceptance_criteria": [
#             "User can pay with credit card, PayPal, or Apple Pay.",
#             "Payment confirmation appears within 2 seconds."
#         ],
#         "feedback_applied": "Added PayPal/Apple Pay per feedback."
#     }]
# """

In [7]:
revised_user_stories_system_prompt = """
# **ROLE & OBJECTIVE**  
You are an expert Agile Product Owner. Your task is to analyze the user stories based on user feedback and refine user stories with existing user stories and return the output in the JSON format only. 

---

TASK BREAKDOWN:  
1. Analyze `user_story` and `user_feedback`.  
2. Identify gaps and improve clarity.  
3. Ensure user stories follow:  
   - "As a [user], I want [goal], so that [value]."  
   - **2 to 4** clear, testable acceptance criteria.  

---

**DESIRED OUTPUT TEMPLATE IN LIST OF JSON**  
```json
[
    {  
        "story_id": "US-001",  
        "title": "Manage Shopping Cart",  
        "description": "As a shopper, I want to modify my cart before checkout, so that I can finalize my purchase conveniently.",  
        "acceptance_criteria": [  
            "User can add/remove items.",  
            "User can update item quantity.",  
            "Cart updates reflect in real-time.",  
            "User can see the total price of the cart."
        ]  
    }  
]
```

---

GUIDELINES:  
✅ Ensure stories are clear, independent, and testable.  
✅ Incorporate feedback without losing business goals.  
✅ Acceptance criteria must be **2 to 4** points.  
🚫 Avoid unnecessary technical details or vague requirements. 
 
"""

In [8]:
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import JsonOutputParser 
output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template="{system_prompt} \n {format_instruction} \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
    partial_variables={"format_instruction" : output_parser.get_format_instructions()}
)

prompt

PromptTemplate(input_variables=['human_query', 'system_prompt'], input_types={}, partial_variables={'format_instruction': 'Return a JSON object.'}, template='{system_prompt} \n {format_instruction} \n {human_query} \n')

In [9]:
def generate_user_stories_from_llm(project_title, project_description, requirements):
    user_query =  f"Create a user stories for the this project title: {project_title} and description: {project_description} and requirements: {requirements}"
    chain = prompt | llm | output_parser
    response = chain.invoke({"system_prompt" : generate_user_stories_system_prompt, "human_query" : user_query})
    return response

In [10]:
user_stories = generate_user_stories_from_llm(project_title, project_description, requirements_string)
user_stories

[{'story_id': 'US-001',
  'title': 'Secure Account with Multi-Factor Authentication',
  'description': 'As a user, I want to secure my account with multi-factor authentication, so that my financial information is protected from unauthorized access.',
  'acceptance_criteria': ['User can enable biometric authentication (fingerprint or facial recognition).',
   'User can set up and use an MPIN for authentication.',
   'The application prompts for MFA during login and sensitive transactions.']},
 {'story_id': 'US-002',
  'title': 'Link Multiple Bank Accounts',
  'description': 'As a user, I want to link multiple bank accounts to PayMate, so that I can easily manage and transfer funds from different accounts.',
  'acceptance_criteria': ['User can add multiple bank accounts to their profile.',
   'User can select a default bank account for transactions.',
   'User can view a list of all linked bank accounts.']},
 {'story_id': 'US-003',
  'title': 'Transfer Funds via UPI',
  'description': 'A

In [18]:

def revised_user_stories_from_llm(user_stories, user_feedback):
    user_query = f"user_feedback: {user_feedback} and user_stories: {user_stories}"
    chain = prompt | llm | output_parser  # Assuming this is the chain or model call
    response = chain.invoke({"system_prompt": revised_user_stories_system_prompt, "human_query": user_query})
    return response

In [12]:

# def revised_user_stories_from_llm(user_stories, user_feedback):
#     user_query = f"user_feedback: {user_feedback} and user_stories: {user_stories}"
#     chain = prompt | llm | output_parser  # Assuming this is the chain or model call
#     response = chain.invoke({"system_prompt": revised_user_stories_system_prompt, "human_query": user_query})
#     return response

In [20]:
user_feedback = "In above user stories, add a user story for buy insurance from the app and return all the user stories"
revised_user_stories = revised_user_stories_from_llm(user_stories, user_feedback) 
revised_user_stories

[{'story_id': 'US-001',
  'title': 'Secure Account with Multi-Factor Authentication',
  'description': 'As a user, I want to secure my account with multi-factor authentication, so that my financial information is protected from unauthorized access.',
  'acceptance_criteria': ['User can enable biometric authentication (fingerprint or facial recognition).',
   'User can set up and use an MPIN for authentication.',
   'The application prompts for MFA during login and sensitive transactions.']},
 {'story_id': 'US-002',
  'title': 'Link Multiple Bank Accounts',
  'description': 'As a user, I want to link multiple bank accounts to PayMate, so that I can easily manage and transfer funds from different accounts.',
  'acceptance_criteria': ['User can add multiple bank accounts to their profile.',
   'User can select a default bank account for transactions.',
   'User can view a list of all linked bank accounts.']},
 {'story_id': 'US-003',
  'title': 'Transfer Funds via UPI',
  'description': 'A

In [21]:
user_feedback = "In above user stories, add a user story for booking a cab from the app and return all the user stories"
revised_user_stories = revised_user_stories_from_llm(output, user_feedback) 
revised_user_stories

[{'story_id': 'US-001',
  'title': 'Secure Account with Multi-Factor Authentication',
  'description': 'As a user, I want to secure my account with multi-factor authentication, so that my financial information is protected from unauthorized access.',
  'acceptance_criteria': ['User can enable biometric authentication (fingerprint or facial recognition).',
   'User can set up and use an MPIN for authentication.',
   'The application prompts for MFA during login and sensitive transactions.']},
 {'story_id': 'US-002',
  'title': 'Link Multiple Bank Accounts',
  'description': 'As a user, I want to link multiple bank accounts to PayMate, so that I can easily manage and transfer funds from different accounts.',
  'acceptance_criteria': ['User can add multiple bank accounts to their profile.',
   'User can select a default bank account for transactions.',
   'User can view a list of all linked bank accounts.']},
 {'story_id': 'US-003',
  'title': 'Transfer Funds via UPI',
  'description': 'A

In [22]:
user_feedback = "In above user stories, remove the Link Multiple Bank Accounts and Access Instant Micro-Loans' and also correct the story_id numbering"
revised_user_stories2 = revised_user_stories_from_llm(revised_user_stories, user_feedback) 
revised_user_stories2

[{'story_id': 'US-001',
  'title': 'Secure Account with Multi-Factor Authentication',
  'description': 'As a user, I want to secure my account with multi-factor authentication, so that my financial information is protected from unauthorized access.',
  'acceptance_criteria': ['User can enable biometric authentication (fingerprint or facial recognition).',
   'User can set up and use an MPIN for authentication.',
   'The application prompts for MFA during login and sensitive transactions.']},
 {'story_id': 'US-002',
  'title': 'Transfer Funds via UPI',
  'description': 'As a user, I want to transfer funds instantly using UPI, so that I can quickly and easily send money to others.',
  'acceptance_criteria': ['User can initiate fund transfers using UPI IDs or QR codes.',
   'User receives confirmation of successful fund transfers.',
   'User can view their transaction history for UPI transfers.']},
 {'story_id': 'US-003',
  'title': 'Pay Utility Bills',
  'description': 'As a user, I want